### <center> **Analysis of COVID-19 Deaths in the United States** </center>

**<center> Anirud Kashyap </center>**


#### Part 1: Data Collection

The first step in the data science pipline is collecting data that is related to the topic this tutorial is focusing on: analyzing COVID-19 deaths and how other various factors such as age group and pre-existing medical conditions may have contributed. Given the significant amount of misinformation that was taking place during this time period, it's important to always consider the validity of data, especially when it involves health. 

The dataset using during this tutorial was obtained through this [source](https://github.com/owid/covid-19-data/tree/master). Fortunately, upon examination of the individual datasets within the repository, the data were obtained from the respective country's governmental organization that was collecting the COVID-related data as seen [here](https://github.com/owid/covid-19-data/blob/master/public/data/testing/covid-testing-latest-data-source-details.csv).

All of the individual datasets were stored in the convienent form of a comma-separated value file.

**Imports & Other Misc.**

In [218]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

We have three datasets that contain our data of interest, which are stored in a comma-separated value file. Let's go ahead and load each one into the respective Pandas dataframe, which offers access to hundreds of useful functions/operations that can be used on the dataframes.

In [219]:
vaccinations = pd.read_csv('/Users/anirudkashyap/Downloads/Lecture/New Tutorial/vaccinations.csv')
hospitalizations = pd.read_csv('/Users/anirudkashyap/Downloads/Lecture/New Tutorial/covid-hospitalizations.csv')
deaths = pd.read_csv('/Users/anirudkashyap/Downloads/Lecture/New Tutorial/total_deaths.csv')

#### Part 2: Data Processing

The purpose of this tutorial is to analyze COVID-19 deaths that have taken place on the global scale. The datasets obtained the first part of the data science pipeline contain many useful features and are formatted in a certain way. However, some of these features add no value, and the way the dataset is set up will hinder our ability to analyze the large datasets. 

In this part, we will process the data by cleaning it up, reorganizing it, and using an imputation technique mentioned later below so that we are able to carry out the remaining stages of the data science pipeline.

Our end goal is to have one dataframe with all of the respective features combined and no missing data.

Let's go ahead and define some helper functions that will be useful in this step of the pipeline.

This helper function takes in a dataframe (df) and a list of features (columns of the dataframe) that are considered unnecessary and need to be removed.

In [220]:
def remove_unnecessary_features(df, features):
    # Drops the features along axis=1
    return df.drop(features, axis=1)

This helper function removes observations in a dataframe that contain a value in a particular feature that will not be used in our data analysis.

In [221]:
def remove_unnecessary_observations(df, feature_name, feature_values):
    
    # Removes all feature_values from the feature feature_name
    df = df[~df[feature_name].isin(feature_values)]

    # Returns the newly filtered dataframe
    return df

This helper function performs a mean imputation technique by filling in the missing np.nan observations with the mean value.

In [222]:
def perform_mean_imp(df, feature_name):
    df[feature_name].fillna(df[feature_name].mean(), inplace=True)

This helper function checks to see if the dataframe `df` has any missing values in the feathre `feature_name`.

In [223]:
def missing_values_present(df, feature_name):
    return df[feature_name].isnull().values.any()

Now that we have loaded the datasets, let's see what are the features for each dataframe.

In [224]:
vaccinations.columns.to_list()

['location',
 'iso_code',
 'date',
 'total_vaccinations',
 'people_vaccinated',
 'people_fully_vaccinated',
 'total_boosters',
 'daily_vaccinations_raw',
 'daily_vaccinations',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_hundred',
 'daily_vaccinations_per_million',
 'daily_people_vaccinated',
 'daily_people_vaccinated_per_hundred']

In [225]:
hospitalizations.columns.to_list()

['entity', 'iso_code', 'date', 'indicator', 'value']

In [226]:
deaths.columns.to_list()[1:5]

['World', 'Afghanistan', 'Africa', 'Albania']

Since we are interested in the total number of 'x', and not daily in the case of the vaccinations dataset contains, let's go ahead and remove those features and only keep the features that keep track of the 'total'.

In [227]:
features_to_remove = [
    'people_vaccinated', 
    'people_fully_vaccinated', 
    'daily_vaccinations_raw', 
    'daily_vaccinations', 
    'daily_vaccinations_per_million',
    'daily_people_vaccinated',
    'daily_people_vaccinated_per_hundred'
]
upd_vaccinations = remove_unnecessary_features(vaccinations, features_to_remove)
upd_vaccinations.columns.to_list()

['location',
 'iso_code',
 'date',
 'total_vaccinations',
 'total_boosters',
 'total_vaccinations_per_hundred',
 'people_vaccinated_per_hundred',
 'people_fully_vaccinated_per_hundred',
 'total_boosters_per_hundred']

Great. Now, as we can see above, the first two datasets contain all of the related data by country and its identifier -- the iso_code. However, the last dataset `deaths`, which contains all of the COVID-19 related deaths, simply has the country name but no iso_code. For the purposes of this tutorial, we will refer to the iso_code in the code, which will allow us to make easier generalizations for each country.

However, before doing that, it's important to note that the deaths dataframe is currently **NOT** tidy in the sense that each country is a feature and each observation is the number of deaths that may have taken place at a given day, as seen below.

In [228]:
deaths.head(5)

,date,World,Afghanistan,Africa,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,...,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Wallis and Futuna,Yemen,Zambia,Zimbabwe
0,2020-01-05,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-06,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-07,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-08,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-09,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's go ahead and "melt" this dataset to make it tidy. Before we melt it, let's go ahead and drop the 'World' column as its data will not be used in this tutorial.

In [229]:
# Removes the 'World' column on the deaths dataframe by calling
# the helper function
deaths = remove_unnecessary_features(deaths, 'World')

# Melts the death dataset, returning a tidy form.
upd_deaths = pd.melt(deaths, id_vars='date', var_name='country', value_name='value')
upd_deaths.head(10)

,date,country,value
0,2020-01-05,Afghanistan,NaN
1,2020-01-06,Afghanistan,NaN
2,2020-01-07,Afghanistan,NaN
3,2020-01-08,Afghanistan,NaN
4,2020-01-09,Afghanistan,NaN
5,2020-01-10,Afghanistan,NaN
6,2020-01-11,Afghanistan,NaN
7,2020-01-12,Afghanistan,NaN
8,2020-01-13,Afghanistan,NaN
9,2020-01-14,Afghanistan,NaN


Great, now all of our dataframes look relatively the same. 

We are interested in the total amount in a feature for a given country. So now let's go ahead and sum up all of the features so that now each country will have its own row and the respective value for the relevant features. 

In [230]:
vaccinations.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,33.0,1367.0,0.003
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,33.0,1367.0,0.003
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,33.0,1367.0,0.003
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,33.0,1367.0,0.003


In [231]:
# Removes the date feature to avoid issues with grouping. However, this
# vaccination dataframe (inc. the date col.) is used in the later part
# of this tutorial.
vaccinations_no_date = remove_unnecessary_features(upd_vaccinations, 'date')
vaccinations_clean = vaccinations_no_date.groupby(['location', 'iso_code']).sum().reset_index()

# Removes the date feature to avoid issues with grouping. However, this
# vaccination dataframe (inc. the date col.) is used in the later part
# of this tutorial.
hospitalizations_no_date = remove_unnecessary_features(hospitalizations, 'date')
hospitalizations_clean = hospitalizations_no_date.groupby(['entity', 'iso_code', 'indicator']).sum().reset_index()

**Final Result:**

In [232]:
vaccinations_clean.head(5)

,location,iso_code,total_vaccinations,total_boosters,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,Afghanistan,AFG,1.659147e+09,9.537989e+07,4034.03,3497.69,3267.78,231.94
1,Africa,OWID_AFR,4.915181e+11,4.002014e+10,34450.57,22053.53,17463.60,2804.86
2,Albania,ALB,3.953643e+08,1.828942e+07,13910.00,6860.83,5856.79,643.54
3,Algeria,DZA,2.983973e+08,3.905993e+06,664.55,355.30,263.07,8.68
4,Andorra,AND,5.846260e+06,1.014711e+06,7322.18,3167.77,2620.12,1270.86


In [233]:
hospitalizations_clean.head(5)

,entity,iso_code,indicator,value
0,Algeria,DZA,Daily ICU occupancy,16593.000
1,Algeria,DZA,Daily ICU occupancy per million,369.523
2,Argentina,ARG,Daily ICU occupancy,1898676.000
3,Argentina,ARG,Daily ICU occupancy per million,41719.682
4,Australia,AUS,Daily ICU occupancy,97483.000


Before we do the same thing to the `upd_deaths` dataframe, there are many missing values that we need to handle before summing it up. However, given the nature of this dataset, we cannot simply impute a value for the missing observations, as it appears tests were not being done at that time (before COVID-19 became a global pandemic). 

And so, as a result, those observations will be dropped.

In [234]:
upd_deaths = remove_unnecessary_observations(upd_deaths, 'value', [np.nan]).reset_index()
upd_deaths_no_date = remove_unnecessary_features(upd_deaths, ['date', 'index'])
upd_deaths_clean = upd_deaths_no_date.groupby('country').sum().reset_index()

**Final Result:**

In [235]:
upd_deaths_clean.head()

,country,value
0,Afghanistan,8530506.0
1,Africa,276122277.0
2,Albania,3891188.0
3,Algeria,7590429.0
4,American Samoa,24163.0


Now that all of the datasets are cleaned up, let's go ahead and merge them all together (using an inner join) into one master dataframe that will be used for our data anlysis in the later stages of the data science pipeline.

In [236]:
# Merges both dataframes through inner join
master_df_one = pd.merge(vaccinations_clean, hospitalizations_clean, left_on='location', right_on='entity', how='inner')
master_df_one.head()

,location,iso_code_x,total_vaccinations,total_boosters,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,entity,iso_code_y,indicator,value
0,Algeria,DZA,2.983973e+08,3.905993e+06,664.55,355.30,263.07,8.68,Algeria,DZA,Daily ICU occupancy,16593.000
1,Algeria,DZA,2.983973e+08,3.905993e+06,664.55,355.30,263.07,8.68,Algeria,DZA,Daily ICU occupancy per million,369.523
2,Argentina,ARG,9.376212e+10,2.363824e+10,206023.97,83254.30,65530.28,51940.04,Argentina,ARG,Daily ICU occupancy,1898676.000
3,Argentina,ARG,9.376212e+10,2.363824e+10,206023.97,83254.30,65530.28,51940.04,Argentina,ARG,Daily ICU occupancy per million,41719.682
4,Australia,AUS,1.921571e+10,3.560549e+09,73405.65,19324.18,21222.67,13601.55,Australia,AUS,Daily ICU occupancy,97483.000


Now let's go ahead and clean up this new master dataframe before merging the `upd_deaths_clean` dataframe.

In [237]:
master_df_one = remove_unnecessary_features(master_df_one, ['entity', 'iso_code_y'])
master_df_one = master_df_one.rename(columns={'iso_code_x':'iso_code', 'value':'hospitalizations'})

Let's go ahead and repeat the same for the `upd_deaths_clean` dataframe.

In [238]:
upd_deaths_clean.head()

,country,value
0,Afghanistan,8530506.0
1,Africa,276122277.0
2,Albania,3891188.0
3,Algeria,7590429.0
4,American Samoa,24163.0


In [240]:
master_df = pd.merge(master_df_one, upd_deaths_clean, left_on='location', right_on='country', how='inner')
master_df = remove_unnecessary_features(master_df, ['country'])
master_df = master_df.rename(columns={'value':'deaths'})

In [243]:
master_df.head()

,location,iso_code,total_vaccinations,total_boosters,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,indicator,hospitalizations,deaths
0,Algeria,DZA,2.983973e+08,3.905993e+06,664.55,355.30,263.07,8.68,Daily ICU occupancy,16593.000,7590429.0
1,Algeria,DZA,2.983973e+08,3.905993e+06,664.55,355.30,263.07,8.68,Daily ICU occupancy per million,369.523,7590429.0
2,Argentina,ARG,9.376212e+10,2.363824e+10,206023.97,83254.30,65530.28,51940.04,Daily ICU occupancy,1898676.000,145850302.0
3,Argentina,ARG,9.376212e+10,2.363824e+10,206023.97,83254.30,65530.28,51940.04,Daily ICU occupancy per million,41719.682,145850302.0
4,Australia,AUS,1.921571e+10,3.560549e+09,73405.65,19324.18,21222.67,13601.55,Daily ICU occupancy,97483.000,15034638.0
